In [217]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import logging
import re

from chemdataextractor import Document
from chemdataextractor.model import Compound
from chemdataextractor.doc import Paragraph, Heading

from chemdataextractor.parse.cem import cem, chemical_label, lenient_chemical_label, solvent_name
from chemdataextractor.parse.common import lbrct, dt, rbrct
from chemdataextractor.utils import first
from chemdataextractor.model import Compound
from chemdataextractor.parse.actions import merge, join
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.parse.elements import W, I, R, Optional, Any, OneOrMore, Not, ZeroOrMore

In [218]:
d = Document(
    Paragraph(u'adsorption enthalpy of Co2 240 JMOL-1') 
    
)

In [219]:
d

<Document: 1 elements>

In [220]:
d.records.serialize()


[]

In [221]:
from chemdataextractor.model import BaseModel, StringType, ListType, ModelType

class AdsorptionEnthalpy(BaseModel):
    prefix= StringType()
    value = StringType()
    units = StringType()
    adsorbate = StringType(contextual=True)
    adsorbent = StringType(contextual=True)
    temperature = StringType(contextual=True)
    temperature_units = StringType(contextual=True)
Compound.adsorption_enthalpies = ListType(ModelType(AdsorptionEnthalpy))

In [222]:
 #add allowances for whitespace, sentence order, /, . or space between derived unit components, absorbate , heat, enthalpy/heat of
    #ΔHr is too general, can it be parsed as a label? might be inferred sometimes, can't have so many optionals? more flexibility in units and symbols, test on real papers, have cem recogniseable from heading or from text
#add ads or adsorption subscript
#add contextual parsers- test if T orks, aso need adsorbate and P
#make MOF recogniseable in para
#need to allow for 'enthalpy of co2 adsorption'
import re
from chemdataextractor.parse.cem import cem, chemical_label, lenient_chemical_label, solvent_name
from chemdataextractor.parse import R, I, W, Optional, merge
import enum
import sre_compile
import sre_parse
import functools

delim = R('^[,:;\.\[\]\(\)\{\}/]$').hide()
delim = R('^[;:,\./]$').hide()
#prefix = ((W(u'ΔHr')) |Optional(I(u'heat')|I(u'enthalpy') + I(u'of')) + Optional( I(u'adsorption') + I(u'heat')|I(u'enthalpy'))).hide()
#this prefix works for most but needs to be more restirctuive
#prefix = ((W(u'ΔHr')) |(I(u'heat')|I(u'enthalpy') + I(u'of') +  I(u'adsorption')) |(I(u'adsorption') + I(u'heat')|I(u'enthalpy'))).hide()
#trying to copy mp prefix = (Optional(I('a')).hide() + (Optional(lbrct) + W(u'ΔHr') + Optional(rbrct))|  ((I('enthalpy') | I('heat')+I('of') +I('adsorption') | (I('adsorption') + (I('enthalpy') | I('heat')| I('range')))).hide() + Optional(lbrct + W('Tm') + rbrct) + Optional(W('=') | I('of') | I('was') | I('is') | I('at')).hide() + Optional(I('in') + I('the') + I('range') + Optional(I('of')) | I('about')).hide()
#this prefix works but needs to be less restrictive wrt of/for at end and cems
#(cem | chemical_label | lenient_chemical_label)
##prefix = (((W(u'ΔHr')) |(I(u'heat')|I(u'enthalpy') + I(u'of') +  I(u'adsorption')) |(I(u'adsorption') + I(u'heat')|I(u'enthalpy')))).hide()
#this one doesn't parse heat of adsorption
#prefix = (((W(u'ΔHr')) |(I(u'heat')|I(u'enthalpy') + I(u'of') +  W(u'adsorption')))).hide()#this matches with just heat
symbol_prefix = (Optional(cem) +W(u'ΔHads')+Optional(I('of')) ).hide()#this matches with just heat
of_prefix = Optional(cem) +(I(u'heat')|I(u'enthalpy')) + I(u'of') + Optional(cem) + I(u'adsorption') + Optional(I('of')) 
standard_prefix = Optional(cem) + I(u'adsorption') + (I(u'heat')|I(u'enthalpy')) 
standard_prefix_of = standard_prefix + Optional(I('of')|I(u'for')) 
prefix_cem_after = symbol_prefix|of_prefix|standard_prefix_of + Optional(cem)
prefix_cem_before = Optional(cem)+ symbol_prefix|of_prefix|standard_prefix_of 
prefix = (prefix_cem_after|prefix_cem_before)
prefix_of= prefix + Optional(I('of')) +  Optional(I('was')) 
#this combined prefix matches all of them without instances of cems
unit_spaces = ((I('kj') +Optional(I(u'/'))+ I('mol-1'))|(I('j') +Optional(I(u'/'))+ I('mol-1')))(u'units').add_action(merge)
unit_nospaces = (Optional(I(u'k'))+I(u'jmol-1')|I(u'kjmol-1'))(u'units').add_action(merge)
unit_slashes = ((I('kj') +(I(u'/'))+ I('mol'))|(I('j') +(I(u'/'))+ I('mol')))(u'units').add_action(merge)
units = unit_spaces|unit_nospaces|unit_slashes
#units = (I(u'kj') + I('mol-1'))(u'units').add_action(merge)
value = R(u'^\d+(\.\d+)?$')(u'value')
ads_nodescriptor = (Optional(cem)+(prefix_of|prefix)+ Optional(I('of')|I(u'for'))+Optional(cem)+ Optional(I('of'))+ Optional(I('was')) + Optional(lbrct).hide() + value + units+ Optional(rbrct).hide())(u'ads')
ads_descriptor = ads_nodescriptor + ZeroOrMore(Not(ads_nodescriptor) + Not(cem) +Any())+cem
ads = (ads_nodescriptor|ads_descriptor)(u'ads')

In [223]:
#chemisorption/physisorption
#(ad/de)sorption?
#- some papers differentiate between componenets of overall adsorption enthalpy,
#but this may be too complex to add
#the adsorbate may be a componenet of the unit e.g. KJ/kmol CO2

In [224]:
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first

class AdsParser(BaseParser):
    root = ads

    def interpret(self, result, start, end):
        compound = Compound(
            adsorption_enthalpies=[
                AdsorptionEnthalpy(
                    value=first(result.xpath('./value/text()')),
                    units=first(result.xpath('./units/text()'))
                )
            ]
        )
        cem_el = first(result.xpath('./cem'))
        if cem_el is not None:
            compound.names = cem_el.xpath('./name/text()')
            compound.labels = cem_el.xpath('./label/text()')
        yield compound

In [225]:
Paragraph.parsers = [AdsParser()]

In [226]:
d = Document(
    Heading(u'Synthesis of MOF-5 (3a)'),
    Paragraph(u'The procedure was followed to yield a pale yellow solid (adsorption enthalpy was 240 KJMOL-1)')
)

d.records.serialize()

[{'names': ['MOF-5'],
  'labels': ['3a'],
  'roles': ['product'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'KJMOL-1'}]}]

In [227]:
e = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u' ΔHads was 240 kJMOL-1')
)
e.records.serialize()

[{'names': ['CD-MOF-2'],
  'labels': ['3a'],
  'roles': ['product'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'kJMOL-1'}]}]

In [228]:
e = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'H2 ΔHads was 240 J/MOL')
)
e.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['H2'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'J/MOL'}]}]

In [229]:
e = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'carbon dioxide ΔHads was (240 kJ/ MOL)')
)
e.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['carbon dioxide'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'kJ/MOL'}]}]

In [230]:
g = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u' methanol heat of adsorption was 240 kJ MOL-1')
)
g.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'kJMOL-1'}]}]

In [231]:
g = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u' heat of adsorption of 240 JMOL-1')
)
g.records.serialize()

[{'names': ['CD-MOF-2'],
  'labels': ['3a'],
  'roles': ['product'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'JMOL-1'}]}]

In [232]:
i = Document(
    Paragraph(u'adsorption heat was 240 JMOL-1')
)
i.records.serialize()

[{'adsorption_enthalpies': [{'value': '240', 'units': 'JMOL-1'}]}]

In [233]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'CD-MOF-2 adsorption heat for methanol of 240 JMOL-1')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'JMOL-1'}]}]

In [234]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'adsorption heat for methanol was 240 JMOL-1')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'JMOL-1'}]}]

In [235]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'methanol heat of adsorption of 240 KJMOL-1')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'KJMOL-1'}]}]

In [236]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of adsorption of methanol of 240 KJMOL-1')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'KJMOL-1'}]}]

In [237]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of adsorption for methanol of 240 KJMOL-1')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'KJMOL-1'}]}]

In [238]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of methanol adsorption 240 KJMOL-1')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'KJMOL-1'}]}]

In [239]:
h = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'methanol adsorption heat 240 JMOL-1')
)
h.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['methanol'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'JMOL-1'}]}]

In [240]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of Co2 adsorption 240 KJMOL-1')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']},
 {'names': ['Co2'],
  'adsorption_enthalpies': [{'value': '240', 'units': 'KJMOL-1'}]}]

In [241]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of co2 adsorption 240 KJMOL-1')
)
i.records.serialize()
#NB only parsed if co2 is capitalised, can you make cem case insensitive?

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']}]

In [242]:
i = Document(
    Paragraph(u'heat of MOF-5 adsorption 240 KJMOL-1')
)
i.records.serialize()
#it doesn't parse the MOF name in the paragraph 
#- although the MOF shouldn't be in this position as an adsorbate, 
#it will need to recognise it as a CEM for the parser to recognise absorbents and adsorbates, 
#we may have to add MOFs to the list of cems or train the CRF chemical entity recogniser

[]

In [243]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of water adsorption 240 KJMOL-1')
)
i.records.serialize()
#it doesn't recognise water as a CEM, 
#some papers investigate MOF adsorption of water, so this needs to be resolved

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']}]

In [213]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'enthalpy  240 KJMOL-1')
)
i.records.serialize()
#this is a test that shouldn't return a value to make sure parser isn't too lenient

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']}]

In [214]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of methanol 240 KJMOL-1')
)
i.records.serialize()
#this is a test that shouldn't return a value to make sure parser isn't too lenient

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']}]

In [200]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'ΔHr for methanol of 240 JMOL-1')
)
i.records.serialize()
#this is a test that shouldn't return a value to make sure parser isn't too lenient
#sometimes ΔHr or #sometimes Δh are contextually of adsorption, need to add this somehow, could search for nearby instances of types of enthalpy descriptors, or the one nearest the first? most likely nearest to that instance
#could maybe use this? https://www.regular-expressions.info/near.html

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']}]

In [215]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'adsorption 240 KJMOL-1')
)
i.records.serialize()
#this is a test that shouldn't return a value to make sure parser isn't too lenient

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']}]

In [216]:
i = Document(
    Heading(u'Synthesis of CD-MOF-2 (3a)'),
    Paragraph(u'heat of adsorption of methanol of 240 KJMOL')
)
i.records.serialize()

[{'names': ['CD-MOF-2'], 'labels': ['3a'], 'roles': ['product']}]